In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import dash
from dash.dependencies import Input,Output
from urllib.request import urlopen
import json
from dash import Dash, dcc, html, dash_table
import plotly.express as px
from jupyter_dash import JupyterDash 
import statsmodels.formula.api as smf
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn import metrics

df = pd.read_csv("../../../../data/superdf_vijay.csv")
app = JupyterDash(__name__)

In [2]:
# replacing nan with zero in the entire dataframe
# renaming the column names (spaces in between the column names are replaced with '_')

df.fillna(0)

df_columns_rename = df.columns
df_columns_rename = [c. replace(' ', '_') for c in df.columns]
df = df.set_axis(df_columns_rename, axis = 1, inplace=False)
df

State  County_Code  Opiod_Dispensing_Rate                 County  \
0       AL         1001                  101.3     Autauga County, AL   
1       AL         1003                   67.6     Baldwin County, AL   
2       AL         1005                   27.2     Barbour County, AL   
3       AL         1007                   21.0        Bibb County, AL   
4       AL         1009                   23.7      Blount County, AL   
...    ...          ...                    ...                    ...   
2522    WY        56033                   33.5    Sheridan County, WY   
2523    WY        56037                   70.0  Sweetwater County, WY   
2524    WY        56039                   54.6       Teton County, WY   
2525    WY        56041                   59.5       Uinta County, WY   
2526    WY        56043                   46.7    Washakie County, WY   

      Population  Deaths  Norm_Deaths  State_FIPS_Code  County_FIPS_Code  \
0        1087149      83          7.6                1                 0   
1        3758097     535         14.2                1                 1   
2         574442      14          2.4                1                 3   
3         464942      56         12.0                1                 5   
4        1171455     168         14.3                1                 7   
...          ...     ...          ...              ...               ...   
2522      598998      61         10.2               47               103   
2523      873221     123         14.1               47               105   
2524      440125      13          3.0               47               107   
2525      426347      61         14.3               47               109   
2526      172114      14          8.1               47               111   

      5-digit_FIPS_Code  ... Male_population_18-44_raw_value  \
0                  1000  ...                             NaN   
1                  1001  ...                             NaN   
2                  1003  ...                             NaN   
3                  1005  ...                             NaN   
4                  1007  ...                             NaN   
...                 ...  ...                             ...   
2522              47103  ...                             NaN   
2523              47105  ...                             NaN   
2524              47107  ...                             NaN   
2525              47109  ...                             NaN   
2526              47111  ...                             NaN   

     Male_population_45-64_raw_value  Male_population_65+_raw_value  \
0                                NaN                            NaN   
1                                NaN                            NaN   
2                                NaN                            NaN   
3                                NaN                            NaN   
4                                NaN                            NaN   
...                              ...                            ...   
2522                             NaN                            NaN   
2523                             NaN                            NaN   
2524                             NaN                            NaN   
2525                             NaN                            NaN   
2526                             NaN                            NaN   

      Total_male_population_raw_value  Female_population_0-17_raw_value  \
0                                 NaN                               NaN   
1                                 NaN                               NaN   
2                                 NaN                               NaN   
3                                 NaN                               NaN   
4                                 NaN                               NaN   
...                               ...                               ...   
2522                              NaN                               

In [3]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
 
app.layout = html.Div([
    html.H1('Opioid Mortality dashboard', style={'textAlign': 'center'}),
    html.Div(children='SIMPLE INTERACTIVE DASHBOARD', style={'textAlign': 'center'}),
    html.Br(),
    dcc.Dropdown(id='dropdown', options=[{'label': x, 'value': x} for x in df.State.unique()], value='NC', multi=False, style= {'width': "42%"}),
    html.Br(),
    
    html.Div([
            html.Label("Linear Or Log"),
            dcc.RadioItems(id="linearlog1",
                options=[
                    {"label": "Linear", "value": "Linear"},
                    {"label": "Log", "value": "Log"}
                ],
                value="Linear", labelStyle={"display":"block"},inline=True)]),
    html.Br(),
    dcc.Dropdown(id='dropdown2', options=[{'label': y, 'value' : y} for y in list(df.columns)], value = 'Opiod_Dispensing_Rate', multi=False, style={'width': "50%"}),
    html.Br(),
    
    html.Div([
            html.Label("Linear Or Log"),
            dcc.RadioItems(id="linearlog2",
                options=[
                    {"label": "Linear", "value": "Linear"},
                    {"label": "Log", "value": "Log"}
                ],
                value="Linear", labelStyle={"display":"block"}, inline=True)]),
    html.Br(),
    html.Div([
            html.Label("Linear Or Non-Linear Regression"),
            dcc.RadioItems(id="linearnonlinear",
                options=[
                    {"label": "Linear regression", "value": "Linear regression"},
                    {"label": "Non-Linear regression", "value": "Non-Linear regression"}
                ],
                value="Linear regression", labelStyle={"display":"block"}, inline=True)]),
    html.Br(),
    html.Div([
            html.Label("Graph or Map"),
            dcc.RadioItems(id="Graph_or_Map",
                options=[
                    {"label": "Graph", "value": "Graph"},
                    {"label": "Map", "value": "Map"}
                ],
                value="Graph")]),
    html.Br(),
    dcc.Graph(id='Graph'),
    dash_table.DataTable(data=df.to_dict('records'), id = 'datatable', columns = [{"name": i, "id": i} for i in df.columns])
                 ])


@app.callback(
     Output(component_id='Graph',component_property='figure'),
     Input('dropdown','value'),
     Input('linearlog1','value'),
     Input('dropdown2','value'),
     Input('linearlog2','value'),
     Input('linearnonlinear','value'),
     Input('Graph_or_Map', 'value'))

def graph(dropdown, linearlog1, dropdown2, linearlog2, linearnonlinear, Graph_or_Map):
    df_1 = df.copy()
    df_1 = df_1[df_1['State']== dropdown]

    if Graph_or_Map == 'Graph':
        
        if linearnonlinear=='Linear regression':
            fig_1= px.scatter(df_1, x =f'{dropdown2}', y='Norm_Deaths', trendline='ols', trendline_scope='trace', trendline_color_override='green')
            fig_1.update_xaxes(title= dropdown2, type='linear' if  linearlog2 == 'Linear' else 'log')
            fig_1.update_yaxes(title='Norm_Deaths', type='linear' if linearlog1 == 'Linear' else 'log')
            return fig_1

        else:
            fig = px.scatter(df, x=df[f'{dropdown2}'], y='Norm_Deaths', trendline = 'ols', trendline_color_override="black")
            fig.update_layout(legend_title_text = " Regression")

            poly_1 = smf.ols(formula = f'Norm_Deaths ~ 1 + {dropdown2}', data = df).fit()
            x = pd.DataFrame({f'{dropdown2}': np.linspace(df[f'{dropdown2}'].min(), df[f'{dropdown2}'].max(), len(df))})

            list1 = list(poly_1.predict(x))
            list2 = list(x[f'{dropdown2}'])

            fig.add_trace(go.Scatter(x = x[f'{dropdown2}'], y = list1, mode = 'lines', name = 'n = 1',  line_color="black"))

            poly_2 = smf.ols(formula = f'Norm_Deaths ~ 1 + {dropdown2} + I({dropdown2} ** 2.0)',data = df).fit()
            list3 = list(poly_2.predict(x))
            fig.add_trace(go.Scatter(x = x[f'{dropdown2}'], y = list3, mode = 'lines', name = 'n = 2'))

            poly_3 = smf.ols(formula = f'Norm_Deaths ~ 1 + {dropdown2} + I({dropdown2} ** 2.0) + I({dropdown2} ** 3.0)',data = df).fit()
            list4 = list(poly_3.predict(x))
            fig.add_trace(go.Scatter(x = x[f'{dropdown2}'], y = list4, mode = 'lines', name = 'n = 3'))

            fig.update_layout(legend_title_text = "Polynomial Regression")

            return fig
      
    else:
        fig = px.choropleth_mapbox(df_1, geojson= counties, locations='County_Code', color=f'{dropdown2}',
                           color_continuous_scale="Viridis",
                           range_color=(0, df_1[f'{dropdown2}'].max()),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'State':'State'}
                          )
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, height = 1000, width = 1000)
        return fig
    
@app.callback(
     Output(component_id='datatable',component_property='data'),
     Input('dropdown','value'))

def data_table(state):
    df_2 = df.copy()
    df_table = df_2[df_2['State'] == state]
    dict_1 = df_table.to_dict('dict')
    return df_table.to_dict('records')




In [4]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash app running on http://127.0.0.1:8050/
